In [ ]:
import numpy as np
import pandas as pd
from DataProcess import combineSeqData, combineLBPSeqData
from sklearn import model_selection
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.decomposition import PCA

Data = pd.read_csv('pdData.csv')
# # 生成'pdDataNF.csv'，该文件是将funny标签转为smiling
# Data["Face"].replace("funny", "smiling", inplace=True)
# print(list(set(Data['Face'])))
# Data.to_csv('pdDataNF.csv', encoding='utf-8')
DataNF = pd.read_csv('pdDataNF.csv')

X0 = combineSeqData(Data)
y0 = np.array(DataNF['Face'].values)


# X1 27个特征
pca = PCA(n_components=27, svd_solver='auto',
          whiten=True).fit(X0)
X1 = pca.transform(X0)

# X2 LBP
X2 = combineLBPSeqData(Data)

# X3 LBP + PCA 27个特征

pca = PCA(n_components=27, svd_solver='auto',
          whiten=True).fit(X2)
X3 = pca.transform(X2)

num_folds = 10
scoring = 'accuracy'

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

print("The face label of the dataset contains:", list(set(DataNF['Face'])))
for name, data in (["Raw", X0], ["PCA", X1], ["LBP", X2], ["LBP + PCA", X3]):
    clf = SVC(kernel='rbf')
    scores = cross_val_score(clf, data, y0,cv=3)
    msg = "%s NB: %f (%f)" % (name, scores.mean(), scores.std())
    print(msg)

In [ ]:
# from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

print(__doc__)
sc = StandardScaler()
sc.fit(X2)
X6 = sc.transform(X2)

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X6, y0, test_size=0.25, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'C': np.logspace(-6, -3, 4), 'gamma': np.logspace(-6, -3, 4)},
                    {'kernel': ['linear'], 'C': np.logspace(-6, -3, 4)}]
scores = ['precision']
print(123)
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf2 = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score,cv=3
    )
    clf2.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf2.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf2.cv_results_['mean_test_score']
    stds = clf2.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf2.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf2.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()